# VSM BESS optimization 
The purpose of this Python Notebook file is to optimize the parameters of the Virtual Synchronous Machine (VSM) controller topology for a grid forming Battery Energy Storage System (BESS) such that it can mimic the frequency response of a given conventional generator. Each code block in this file represents a key step in the optimization process. All the inputs to this file are entered into the first block of code, so this is the only part of the code that should be modified. Note that a Synchronous Condenser (SC) can be enabled which will replace the inertial response of the BESS. As such, the inertial response of the BESS is disabled if the SC is enabled. 

The parameters that this file optimizes are:
* The droop constant of the frequency controller
* The inertia constant of the frequency controller (only if the SC is disabled, otherwise this value is set to 'N/A')
* The damping constant of the frequency controller

All other parameters are set to constant values.

### Inputs
The following code block contains all the inputs that the user may want to modify. Note the following:
* The information of the generator can be found in the PowerFactory base file and excel results file. 
* The event information is completely arbitrary, but it should be checked that the base case remains stable, and that the BESS stays within its SOC bounds for whatever event is defined.
* The battery pack series cells is also an arbitrary choice, and the default value of 9753 was chosen as it achieves a realistic DC voltage for a battery of this size and results in a parallel cell number that is close to an integer value, so there is negligible inaccuracy due to rounding.
* The VSM BESS and synchronous condenser information can be found in the PowerFactory base file and excel results file. 
* The optimization settings should be tuned to achieve the desired level of accuracy while still converging within a reasonable time frame. Tune the BESS parameters by hand in PowerFactory to be as close as possible to the base case response and choose bounds around the manually tuned values. 
* There is two simulation end times, one for each stage of the optimization. The simulation end time is initially set to 600s so that it can be ensured that the frequency response has reached steady state (sim_stop_time1). The simulation end time can be shortened to 300s once the Quasi Steady State Frequency (QSSF) has been matched (sim_stop_time2), which reduces computation time.
* Setting 'SC_connected' to False will autmatically turn of the SC and enable the inertial response of the BESS. Setting 'SC_connected' to True will autmatically turn on the SC and disable the inertial response of the BESS. 

In [1]:
 # Import packages
import pandas as pd
import simulationfunctions as simfunc
import optimization as opt
import numpy as np
import plotly.express as px
from scipy.optimize import direct, Bounds
from sklearn.metrics import mean_squared_error
import sys

# Information of the generator being replaced by the BESS
proj_name_base = r"\mattm\Base cases\Simple System - BASE (G10)" # Base case file path
gen_data_name = "G 10" # Name of generator in excel data file

# Event information
event = 0 # Event type: "0" = load change, "1" = generator event (under frequency event) - for optimization we use a load change event
load_change = 10 # Step change for load event in %
event_time = 5 # Event time in s
load_model_name = "Load 01.ElmLod" # Name of the model of the load being changed
gen_model_name = "None" # Name of the model of the generator that is swicthed off in a generator event (not used for optimization)

# Battery pack information
BESS_series_cells = 585 # Number of series cells in the battery pack (integer)
BESS_soc0 = 0.5 # Intial SOC of the battery

# VSM BESS information
proj_name_BESS = r"\mattm\VSM cases\Simple System - VSM BESS" # BESS case file path
BESS_data_name = 'VSM BESS' # Name of BESS converter in excel data file
BESS_power_name = 'm:Psum:bus1 in MW' # Name of BESS power column in excel data file
BESS_freq_model = "CM-Virtual Inertia.ElmDsl"  # Name of frequency controller model 
BESS_PWM_model = "VSM BESS.ElmGenstat" # Name of PWM model (for changing power output)
BESS_impedance_model = "CM-Virtual Impedance.ElmDsl" # Name of virtula impedance model
BESS_RPC_model = 'CM-RPC.ElmDsl' # Name of reactive power control model
BESS_CC_model = "CM_CC.ElmDsl" # Name of charge control model
BESS_battery_model = "Battery Model 13Ah.ElmDsl" # Name of battery model

# Synchronous condenser information
SC_connected = False # Set to true if a synchronous condenser is connected in the BESS file
SC_data_name_G03 = 'SynCon G03' # Name of SC replacing G03 in excel data file
SC_data_name_G10 = 'SynCon G10' # Name of SC replacing G10 in excel data file
SC_model_G03 = 'SynCon G03.ElmSym' # Model of SC replacing G03
SC_model_G10 = 'SynCon G10.ElmSym'  # Model of SC replacing G10

# Optimization settings
min_ss_error = 1e-5 # Minimum steady state error value for convergence of the first stage of optimization (setting closer to 0 fits the responses more closely but leads to longer simulation time)
min_rms_error = 1e-4 # Minimum RMS error value for convergence of the second stage of optimization (setting closer to 0 fits the responses more closely but leads to longer simulation time)

# Bounds on optimization variables
droop_lower = 4 # Droop lower bound
droop_upper = 5 # Droop upper bound
inertia_lower = 2 # Inertia constant lower bound (not used if SC is enabled)
inertia_upper = 6 # Inertia constant upper bound (not used if SC is enabled)
damping_lower = 100 # Damping constant lower bound
damping_upper = 200 # Damping constant upper bound

# Simulation setup 
sim_start_time = -100 # Simulation start time (ms)
sim_stop_time1 = 600 # Simulation end time for first optimization stage
sim_stop_time2 = 300 # Simulation time for second optimization stage (can be shortened as steady state freqeuncy has been matched)
digsilent_path = r"C:\Program Files\DIgSILENT\PowerFactory 2023 SP2\Python\3.10" # DIgSILENT files path

# Results information
freq_data_name = "Bus 01" # Name of bus the frequency is being measured from in excel data file
base_results_path =  r"C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\simulation_results_BASE.csv" # Base case results file path
BESS_results_path =  r"C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\simulation_results_BESS.csv" # BESS case results file path
optimal_results_path = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Optimization_results.xlsx' # Optimization results file path
freq_image_path = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\FreqResp.png' # Frequency response image file path
p_image_path = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\GenP.png' # Power output image file path

### Simulation set up and base case results
The following code block opens PowerFactory so that simulations can be conducted. Then the base case is executed, and the results are stored so that the optimization stage can calculate the error. PowerFactory will print below if there are no errors.


In [ ]:
# Open PowerFactory
sys.path.append(digsilent_path) 
import powerfactory as pf
try:
    app = pf.GetApplicationExt()
    print(app) # PowerFactory will print below if there are no errors
except pf.ExitError as error: 
    print(error)
    print('error.code = %d' % error.code)

# Set up base case (define the event)
Rms, oInit, elmres = simfunc.base_case_setup(app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time1, proj_name_base)

# Run base case and save results
dfs_b = simfunc.base_case_sim(app, event_time, base_results_path, freq_data_name, Rms, oInit, elmres)
base_nadir, base_nadirtime, base_rocofapprox, base_ssfreq, base_overshoot, base_overshoottime = simfunc.getNadirROCOFs_csv(dfs_b, event_time, freq_data_name)

### Calculate the BESS parameters 
The battery pack parameters are calculated based on the known cell data and the number of series cells. It is assumed that the BESS needs to have the same MVA rating as the generator it is replacing, so the number of parallel cells needed to achieve this rating can be calculated based on this. This indirectly determines the capacity value of the battery. As the C-rate of these cells is 3C, the capacity will always be 1/3 of the generator rating. The capacity could be chosen arbitrarily, but it is sensible to have the battery have the same maximum power output as the generator it is replacing. 

As the inertia gain represents the value 2H in a generator, the initial value is set to be equal to the 2H, where H inertia of the generator it is replacing. However, this value could be changed arbitrarily and will be optimized later. If the SC is enabled, the inertial response of the BESS is disabled. The damping coefficient is set to a small value of 50 by default to ensure that the response settles by the end of the simulation.  


In [ ]:
# Set up BESS case corresponding to the generator being replaced
if(gen_data_name == "G 03"):
    SC_model = SC_model_G03
    SC_data_name = SC_data_name_G03
    generator_rated_power = 800 # Rating of the generator in MVA
    generator_inertia = 4.475 # Inertia of the generator in s (G10 = 4.2s, G03 = 4.475s)
    gen_setpoint = 250 # Active power setpoint of the generator in MW 
    lv_value = 0.0002
    q_value = 0
    
if(gen_data_name == "G 10"):
    SC_model = SC_model_G10
    SC_data_name = SC_data_name_G10
    generator_rated_power = 1000 # Rating of the generator in MVA
    generator_inertia = 4.2 # Inertia of the generator in s (G10 = 4.2s, G03 = 4.475s)
    gen_setpoint = 250 # Active power setpoint of the generator in MW 
    lv_value = 0.01
    q_value = 500

# Battery pack constants
cell_voltage = 2.228 # nominal voltage of one cell in V
c_rate = 0.5 # C rate of the cell (max current) 
cell_capacity = 13 # Capacity rating of one cell in Ah 

# BESS parameters 
BESS_setpoint = gen_setpoint # BESS power setpoint (set to same as conventional generator it is replacing)
BESS_rated_power =  generator_rated_power # BESS rated power (set to same as conventional generator it is replacing)

# BESS calcualtions
BESS_DC_voltage = BESS_series_cells * cell_voltage # Total DC voltage of the battery pack in V
BESS_par_cells = round((BESS_rated_power*10**6)/ (cell_capacity * c_rate * BESS_DC_voltage)) # Number of parallel cells needed to achieve desired power rating 
BESS_capacity = cell_capacity * BESS_par_cells * BESS_DC_voltage/ (10**6) # Capacity of the battery pack in MWh

# Default values for first stage optimization
damping_value = 50 # Initally set to small value 
if SC_connected:
    inertia_value = 0 # Inertial response in BES will be disabled
else:
    inertia_value = 2*generator_inertia # inertia constant is initally set to equal 2H of the generator

Rms, oInit, elmres = simfunc.VSM_case_setup(app, event, event_time, load_change, lv_value, q_value, SC_connected, load_model_name, gen_model_name, sim_start_time, sim_stop_time1, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_impedance_model, BESS_RPC_model, BESS_CC_model, BESS_freq_model, SC_model)  

# Disable the SC that is not in use
if(gen_data_name == "G 03"):
    simfunc.toggle_SC(app, SC_model_G10, False)
if(gen_data_name == "G 10"):
    simfunc.toggle_SC(app, SC_model_G03, False) 


### Optimization Stage 1: Quasi Steady State Frequency error
As the power rating of the BESS is now fixed, the only parameter that has any effect on the Quasi Steady State Frequency (QSSF) value is the droop constant value. The first stage of the optimization aims to match the QSSF values of the base case and BESS case by finding the optimal droop constant value.   

In [ ]:
# Match SS frequency using droop value
droop_bounds = Bounds(droop_lower, droop_upper) # Set droop bounds
ss_error = direct(opt.VSM_QSSF_objective_func, droop_bounds, args=(inertia_value, damping_value,SC_connected, app, event_time, BESS_freq_model, BESS_results_path, freq_data_name, base_ssfreq, Rms, oInit, elmres), f_min = min_ss_error) 
droop_optimal = ss_error.x[0]
print("Optimal value:", ss_error.x)
print("Number of function evaluations:", ss_error.nfev)

### Shorten simulation time
As the QSSF has been matched, the simulation time can be shortened to speed up the computation time.

In [ ]:
# Store base case results for shorter simulation time 
Rms, oInit, elmres = simfunc.base_case_setup(app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time2, proj_name_base)
dfs_b = simfunc.base_case_sim(app, event_time, base_results_path, freq_data_name, Rms, oInit, elmres)
base_nadir, base_nadirtime, base_rocofapprox, base_ssfreq, base_overshoot, base_overshoottime = simfunc.getNadirROCOFs_csv(dfs_b, event_time, freq_data_name)

# Set up BESS case for shorter simualtion time
Rms, oInit, elmres = simfunc.VSM_case_setup(app, event, event_time, load_change, lv_value, q_value, SC_connected, load_model_name, gen_model_name, sim_start_time, sim_stop_time2, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_impedance_model, BESS_RPC_model, BESS_CC_model, BESS_freq_model, SC_model)

### Optimization Stage 2: Root Mean Square error
The inertia and damping constants determine the "shape" of the frequency response. The second stage of the optimization aims to match the RMS error between the base case and BESS case frequency responses by finding the optimal damping and inertia values.   

In [ ]:
# Minimize RMS error using damping value. 
ID_bounds = Bounds([inertia_lower, damping_lower], [inertia_upper, damping_upper]) # Bounds on the inertia constant and damping values 
rms_error = direct(opt.VSM_RMS_objective_func, ID_bounds, args=(droop_optimal, SC_connected, app, event_time, BESS_freq_model, BESS_results_path, freq_data_name, dfs_b, Rms, oInit, elmres), f_min = min_rms_error, locally_biased=False) 
inertia_optimal, damping_optimal  = rms_error.x

print("Optimal value:", rms_error.x)
print("Number of function evaluations:", rms_error.nfev)

### Store and plot the results
The results of the optimization are stored in an excel file. The power outputs and frequency responses of both the base case and BESS is plotted and exported. If the result (stored in the "Result output" folder) is satisfactory, copy the contents from this folder into the corresponding results folder to ensure that it is not overwritten.

In [ ]:
# Store the optimal values
optimal_solution = [droop_optimal,  inertia_optimal, damping_optimal]
print("Optimal solution:", optimal_solution)

# Run the BESS case with the optimal parameters
droop_value, inertia_value, damping_value = optimal_solution
dfs_BESS = simfunc.VSM_case_sim(droop_value, inertia_value, damping_value, SC_connected, app, event_time, BESS_freq_model, BESS_results_path, freq_data_name, Rms, oInit, elmres)

# Filter data frames to exclude data before t = 0
dfs_BESS_filtered = dfs_BESS[dfs_BESS[("All calculations", "b:tnow in s")] >= 0]
dfs_b_filtered = dfs_b[dfs_b[("All calculations", "b:tnow in s")] >= 0]

# Find the RMS error of the power output responses
if(SC_connected):
    power_rms_error = mean_squared_error(dfs_BESS_filtered[(BESS_data_name,'m:Psum:bus1 in MW')]+ dfs_BESS_filtered[(SC_data_name,'m:Psum:bus1 in MW')], dfs_b_filtered[(gen_data_name,'m:Psum:bus1 in MW')], squared = False) 
else:
    power_rms_error = mean_squared_error(dfs_BESS_filtered[(BESS_data_name,'m:Psum:bus1 in MW')], dfs_b_filtered[(gen_data_name,'m:Psum:bus1 in MW')], squared = False) 
print("Power RMS error:", power_rms_error)

# Find the RMS error of the frequency responses
frequency_rms_error = mean_squared_error(dfs_BESS_filtered[(freq_data_name,"m:fehz in Hz")], dfs_b_filtered[(freq_data_name,"m:fehz in Hz")], squared = False) # Frequency RMS error
print("Frequency RMS error:", frequency_rms_error)

# Export optimal results
optimal_results = pd.DataFrame({
    'Droop constant': [optimal_solution[0]],
    'Inertia constant': [optimal_solution[1]],
    'Damping constant': [optimal_solution[2]],
    'Power RMSE': power_rms_error,
    'Frequency RMSE': frequency_rms_error
})
optimal_results.to_excel(optimal_results_path, index=False) # write the DataFrame to an Excel file

# Generate interactive plots for analysis
fig_Gen_P = simfunc.plot_power_px(dfs_b_filtered, dfs_BESS_filtered, gen_data_name, BESS_data_name, BESS_power_name, SC_data_name, SC_connected)
fig_Freq = simfunc.plot_freq_px(dfs_b_filtered, dfs_BESS_filtered, freq_data_name)

# Generate and save plots for the report based on the desired length of the data
data_length = 100 # Period of the simulation you want to actually plot (seconds)
simfunc.plot_freq(dfs_b_filtered, dfs_BESS_filtered, freq_data_name, freq_image_path, data_length)
simfunc.plot_power(dfs_b_filtered, dfs_BESS_filtered, gen_data_name, BESS_data_name, BESS_power_name, SC_data_name, SC_connected, p_image_path, data_length)
